goal:
=====

multiclass logistic regression with the inner product replaced by a more general kernal function

In [1]:
# import this stuff
import time
import sys
from pylab import *
from scipy import sparse
import numpy as np

import tensorflow as tf

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

In [120]:
def factorize(observed_features,
              labels,
              observed_features_validation,
              labels_validation,
              max_iter=100,
              verbose=False,
              lambda_v=0,
              kernel='linear',
              epsilon=0.0001,
              optimizer=tf.train.AdamOptimizer(),
              seed=12345):

    # Extract info about shapes etc from the training data
    num_items = observed_features.shape[0]
    num_features = observed_features.shape[1]
    num_classes = labels.shape[1]
 
    v = tf.Variable(tf.truncated_normal([num_classes, num_features], stddev=0.2, mean=0, seed=seed), name="hyperplane")
    
    x = tf.placeholder(tf.float32, [1, num_features])
    y = tf.placeholder(tf.float32, [1, num_classes])
    x_norm = tf.nn.l2_normalize(x,dim=0)
    
    if kernel=='linear':
        ip = tf.matmul(v, tf.transpose(x))
    elif kernel=='gaussian':
        ip = tf.reshape(tf.exp(-tf.reduce_sum(tf.square(tf.sub(v, x)), reduction_indices=1)/10), [num_classes, 1])
    else:
        raise Exception("unknown kernel: %s" % kernel)


    pred = tf.nn.softmax(tf.transpose(ip))
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred + 0.0000000001), reduction_indices=1) + # this was causing nans if pred == 0
                          lambda_v*tf.nn.l2_loss(v)) # regularization for v
    
    norm = tf.nn.l2_loss(v)
    optimize = optimizer.minimize(cost)

    init = tf.initialize_all_variables()
    
    with tf.Session() as sess:
        sess.run(init)
        last_cost = 1000000
        for iter in range(0, max_iter):
            avg_cost = 0
            
            for i in range(num_items):
                _, c, n = sess.run([optimize, cost, norm],
                                   feed_dict={x:observed_features[i].reshape(1,num_features), y:labels[i].reshape(1,num_classes)})
                avg_cost += c / num_items
            if verbose:
                print("epoch: %s, cost: %s, norm: %s" % (iter+1, avg_cost, n))

            # check for convergence
            if abs(avg_cost-last_cost)/avg_cost < epsilon:
                break
                
            last_cost = avg_cost
            
        if verbose:
            print("optimization finished")
        # test prediction
        predictions = []
        total_costs = 0
        for i in range(observed_features_validation.shape[0]):
            p, c = sess.run([pred, cost], feed_dict={x:observed_features_validation[i].reshape(1, num_features), y:labels_validation[i].reshape(1, num_classes)})
            predictions.append(p)
            total_costs += c
        return np.array([z[0,:] for z in predictions]), total_costs/observed_features_validation.shape[0], sess.run([norm])

In [121]:
# use this data for now

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

ng = datasets.fetch_20newsgroups (categories=categories, shuffle=True)
encoder = OneHotEncoder(sparse=False)
labels = encoder.fit_transform(ng.target.reshape(-1,1))

tfidf = TfidfVectorizer(decode_error=False, min_df=5)

X_train, X_test, y_train, y_test = train_test_split(ng.data, labels, test_size=.3)
X_train = tfidf.fit_transform(X_train).todense()
X_test = tfidf.transform(X_test).todense()


In [124]:

predictions, test_costs, norm = factorize(X_train, y_train, X_test, y_test, verbose=True, lambda_v=0.001, max_iter=100, kernel='gaussian')
print("cost: %s, norm: %s") % (test_costs, norm)
for i in range(y_train.shape[1]):
    print("class %s AUC: %s") % (i, roc_auc_score(y_test[:,i], predictions[:,i]))
print("overall AUC: %s") % roc_auc_score(y_test, predictions, average="weighted")

epoch: 1, cost: 1.42283913521, norm: 11.4784
epoch: 2, cost: 1.37272652869, norm: 7.24813
epoch: 3, cost: 1.36564107822, norm: 6.92574
epoch: 4, cost: 1.36451845618, norm: 7.02357
epoch: 5, cost: 1.36431197119, norm: 7.13067
epoch: 6, cost: 1.36426895355, norm: 7.2063
optimization finished
cost: 1.36253122348, norm: [7.2149992]
class 0 AUC: 0.975390709789
class 1 AUC: 0.952663270032
class 2 AUC: 0.994666502559
class 3 AUC: 0.899048969224
overall AUC: 0.956998434202
